In [1]:
import unidecode
import pandas as pd
import numpy as np
import re
import nltk
nltk.download("stopwords")
from unidecode import unidecode
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
import pandas as pd
from bs4 import BeautifulSoup
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf

pd.set_option('display.max_colwidth', 200)

#carregando arquivo
df = pd.read_excel(r"D:\Documents\ESTUDOS\POS GRADUACAO\TCC\base_nps_completa_original.xlsx")

#limpando pesquisas em outro idioma
df = df[df['surveyId'] == 'NPS Zendesk Português']
#removendo números
df['comentario_NPS'] = df['comentario_NPS'].apply(lambda x: re.sub(r'\d', '', str(x)))
#removendo "\n" dos comentários
df['comentario_NPS'].replace("\n", '', inplace = True)
#trocando comentários em branco ou sem letras por NONE
df['comentario_NPS'] = df['comentario_NPS'].apply(lambda x: x if re.fullmatch(r".*[a-zA-Z].*", x) != None else None)
df['comentario_NPS'].replace({'', 'NULL','nan'}, None, inplace = True)
#filtrando apenas pesquisas com comentários
df = df[df['comentario_NPS'].isnull() == False]
#transformando tudo em minúsculas
df['comentario_NPS'] = df['comentario_NPS'].apply(lambda x: x.lower())
#filtrando apenas as colunas necessárias
df = df[['metadata_ticket', 'resposta_NPS', 'rqt_tipocliente', 'comentario_NPS']]

#separando dataframes por tipo de ticket
df_internos = df[df['rqt_tipocliente'] == 'INTERNO']
df_externos = df[df['rqt_tipocliente'] != 'INTERNO']

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Renan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\Users\Renan\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
def text_cleaning(text):
    soup = BeautifulSoup(text, "html.parser")
    text = re.sub(r'\[[^]]*\]', '', soup.get_text())
    text = unidecode(text)
    pattern = r"[^a-zA-Z0-9\s,']"
    text = re.sub(pattern, '', text)
    return text

def to_sentiment(rating):
  rating = int(rating)
  if rating < 5:
    return 0
  elif rating == 10:
    return 1

In [47]:
df.rename(columns={'comentario_NPS': 'sentence'}, inplace=True)

df['sentiment'] = df.resposta_NPS.apply(to_sentiment)

df['Cleaned_sentence'] = df['sentence'].apply(text_cleaning).tolist()

#estratificando manualmente
auxdf0 = df[df['sentiment']==0]
auxdf1 = df[df['sentiment']==1].sample(n=len(auxdf0), random_state=42)
final_df = pd.concat([auxdf0, auxdf1], ignore_index=True)


final_df['sentiment'] = final_df['sentiment'].astype(np.int64)
final_df = final_df.sample(frac=1).reset_index(drop=True)

remaining_df = df[~df['metadata_ticket'].isin(final_df['metadata_ticket'])]

train_df, test_df = train_test_split(final_df, test_size=0.3)

# Training data
#Reviews = "[CLS] " +train_df['Cleaned_sentence'] + "[SEP]"
Reviews = train_df['Cleaned_sentence']
Target = train_df['sentiment']
 
# Test data
#test_reviews =  "[CLS] " +test_df['Cleaned_sentence'] + "[SEP]"
test_reviews = test_df['Cleaned_sentence']
test_targets = test_df['sentiment']

x_val, x_test, y_val, y_test = train_test_split(test_reviews,
                                                    test_targets,
                                                    test_size=0.5, 
                                                    stratify = test_targets)

C:\Users\Renan\AppData\Local\Temp\ipykernel_9384\488764235.py:2: MarkupResemblesLocatorWarning:

The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.



In [33]:
#aplicando a tokenizacao em portugues com BERTimbau
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=True)

max_len= 175
# Tokenize and encode the sentences
X_train_encoded = tokenizer.batch_encode_plus(Reviews.tolist(),
                                              padding=True, 
                                              truncation=True,
                                              max_length = max_len,
                                              return_tensors='tf')
 
X_val_encoded = tokenizer.batch_encode_plus(x_val.tolist(), 
                                              padding=True, 
                                              truncation=True,
                                              max_length = max_len,
                                              return_tensors='tf')
 
X_test_encoded = tokenizer.batch_encode_plus(x_test.tolist(), 
                                              padding=True, 
                                              truncation=True,
                                              max_length = max_len,
                                              return_tensors='tf')

#criando modelo
model = TFBertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=2)

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier', 'bert/pooler/dense/bias:0', 'bert/pooler/dense/kernel:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
#treinando
history = model.fit(
    [X_train_encoded['input_ids'], X_train_encoded['token_type_ids'], X_train_encoded['attention_mask']],
    Target,
    validation_data=(
      [X_val_encoded['input_ids'], X_val_encoded['token_type_ids'], X_val_encoded['attention_mask']],y_val),
    batch_size=32,
    epochs=3
)

Epoch 1/3
93/93 [==============================] - 902s 10s/step - loss: 0.2092 - accuracy: 0.9273 - val_loss: 0.1362 - val_accuracy: 0.9669
Epoch 2/3
93/93 [==============================] - 886s 10s/step - loss: 0.0721 - accuracy: 0.9784 - val_loss: 0.1279 - val_accuracy: 0.9716
Epoch 3/3
93/93 [==============================] - 879s 9s/step - loss: 0.0471 - accuracy: 0.9875 - val_loss: 0.1180 - val_accuracy: 0.9748


In [35]:
test_loss, test_accuracy = model.evaluate(
    [X_test_encoded['input_ids'], X_test_encoded['token_type_ids'], X_test_encoded['attention_mask']],
    y_test
)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')

20/20 [==============================] - 55s 3s/step - loss: 0.1045 - accuracy: 0.9779
Test loss: 0.10454543679952621, Test accuracy: 0.9779179692268372


In [36]:

path = 'D:\Documents\ESTUDOS\POS GRADUACAO\TCC'
# Save tokenizer
tokenizer.save_pretrained(path +'/Tokenizer2')
 
# Save model
model.save_pretrained(path +'/Model2')


In [37]:
bert_tokenizer = BertTokenizer.from_pretrained(path +'/Tokenizer2')
 
# Load model
bert_model = TFBertForSequenceClassification.from_pretrained(path +'/Model2')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some layers from the model checkpoint at D:\Documents\ESTUDOS\POS GRADUACAO\TCC\LeIA-master/Model2 were not used when initializing TFBertForSequenceClassification: ['dropout_75']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at D:\Documents\ESTUDOS\POS GRADUACAO\TCC\LeIA-master/Model2.
If your task is similar to the task the model of th

In [38]:

pred = bert_model.predict(
    [X_test_encoded['input_ids'], X_test_encoded['token_type_ids'], X_test_encoded['attention_mask']])
 
# pred is of type TFSequenceClassifierOutput
logits = pred.logits
 
# Use argmax along the appropriate axis to get the predicted labels
pred_labels = tf.argmax(logits, axis=1)
 
# Convert the predicted labels to a NumPy array
pred_labels = pred_labels.numpy()
 
label = {
    1: 'positive',
    0: 'Negative'
}
 
# Map the predicted labels to their corresponding strings using the label dictionary
pred_labels = [label[i] for i in pred_labels]
Actual = [label[i] for i in y_test]
 
print('Predicted Label :', pred_labels[:10])
print('Actual Label    :', Actual[:10])


20/20 [==============================] - 56s 3s/step
Predicted Label : ['positive', 'positive', 'positive', 'Negative', 'Negative', 'positive', 'Negative', 'positive', 'positive', 'positive']
Actual Label    : ['positive', 'positive', 'positive', 'Negative', 'Negative', 'positive', 'Negative', 'positive', 'positive', 'positive']


In [39]:
print("Classification Report: \n", classification_report(Actual, pred_labels))


Classification Report: 
               precision    recall  f1-score   support

    Negative       0.97      0.99      0.98       315
    positive       0.99      0.97      0.98       319

    accuracy                           0.98       634
   macro avg       0.98      0.98      0.98       634
weighted avg       0.98      0.98      0.98       634



In [40]:

def Get_sentiment(Review, Tokenizer=bert_tokenizer, Model=bert_model):
    # Convert Review to a list if it's not already a list
    if not isinstance(Review, list):
        Review = [Review]
 
    Input_ids, Token_type_ids, Attention_mask = Tokenizer.batch_encode_plus(Review,
                                                                             padding=True,
                                                                             truncation=True,
                                                                             max_length=128,
                                                                             return_tensors='tf').values()
    prediction = Model.predict([Input_ids, Token_type_ids, Attention_mask])
 
    # Use argmax along the appropriate axis to get the predicted labels
    pred_labels = tf.argmax(prediction.logits, axis=1)
 
    # Convert the TensorFlow tensor to a NumPy array and then to a list to get the predicted sentiment labels
    pred_labels = [label[i] for i in pred_labels.numpy().tolist()]
    return pred_labels


In [ ]:
#aplicando o modelo em cada um dos comentários que sobraram, nossos alvos
remaining_df['predicted'] = remaining_df['Cleaned_sentence'].apply(lambda x: Get_sentiment(x))

In [57]:
remaining_df.to_excel("output.xlsx")

In [71]:
validacao = pd.read_excel('output.xlsx')

In [113]:
def to_sentiment2(rating):
  rating = int(rating)
  if rating < 7:
    return 'Detrator'
  elif rating > 8:
    return 'Promotor'
  else:
    return 'Neutro'
  
validacao['sentiment'] = validacao.resposta_NPS.apply(to_sentiment2)

In [114]:
validacao

y_actu = pd.Series(validacao['sentiment'].to_list(), name='Actual')
y_pred = pd.Series(validacao['predicted'].to_list(), name='Predicted')
#df_confusion = pd.crosstab(y_actu, y_pred)

df_confusion = pd.crosstab(y_actu, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)
df_confusion

Predicted,['Negative'],['positive'],All
Actual,,,
Detrator,567,31,598
Neutro,533,357,890
Promotor,705,10995,11700
All,1805,11383,13188


In [119]:
def resumidor(rating):
  if rating == 'INTERNO':
    return 'INTERNO'
  else:
    return 'EXTERNO'
  
validacao['tipocliente'] = validacao.rqt_tipocliente.apply(resumidor)

pd.DataFrame(validacao.groupby(['predicted','resposta_NPS', 'tipocliente'])['predicted'].count())

predicted
predicted    resposta_NPS tipocliente           
['Negative'] 5            EXTERNO            349
                          INTERNO             13
             6            EXTERNO            198
                          INTERNO              7
             7            EXTERNO            236
                          INTERNO             14
             8            EXTERNO            268
                          INTERNO             15
             9            EXTERNO            234
                          INTERNO             13
             10           EXTERNO            430
                          INTERNO             28
['positive'] 5            EXTERNO             15
             6            EXTERNO             16
             7            EXTERNO             64
                          INTERNO              1
             8            EXTERNO            280
                          INTERNO             12
             9            EXTERNO            833
                          INTERNO            306
             10           EXTERNO           7840
                          INTERNO           2016

In [126]:
remaining_df.to_excel("Resultados_Finais.xlsx")